In [29]:
import numpy as np
from notebook_video_writer import VideoWriter
import gymnasium as gym

## Create Environment

In [30]:
env = gym.make('Taxi-v3', render_mode = 'rgb_array')
terminated, truncated = False, False
state, info = env.reset()
env_video = []
while not (terminated or truncated):
    action = env.action_space.sample()
    env_video.append(env.render())
    state, reward, terminated, truncated, _ = env.step(action)
    print(state, reward, terminated, truncated)
    
with VideoWriter(fps= 5) as vw:
    for frame in env_video:
        vw.add(frame)

329 -10 False False
229 -1 False False
249 -1 False False
229 -1 False False
229 -10 False False
249 -1 False False
249 -10 False False
249 -10 False False
269 -1 False False
249 -1 False False
269 -1 False False
289 -1 False False
289 -10 False False
289 -10 False False
389 -1 False False
389 -10 False False
389 -10 False False
389 -1 False False
489 -1 False False
489 -10 False False
489 -10 False False
489 -10 False False
389 -1 False False
389 -10 False False
389 -10 False False
289 -1 False False
389 -1 False False
389 -1 False False
389 -10 False False
489 -1 False False
489 -10 False False
489 -10 False False
489 -1 False False
489 -10 False False
489 -1 False False
489 -10 False False
389 -1 False False
489 -1 False False
489 -10 False False
489 -1 False False
469 -1 False False
369 -1 False False
369 -10 False False
369 -10 False False
369 -10 False False
469 -1 False False
369 -1 False False
369 -1 False False
369 -10 False False
389 -1 False False
369 -1 False False
389 -1 F

In [31]:
class Agent:
    def __init__(self, env):
        self.env = env
        self.q_table = np.zeros([env.observation_space.n, env.action_space.n])
        self.alpha = 0.5
        self.gamma = 0.9
        self.epsilon = 0.5
        
    def choose_action(self, state):
        return np.argmax(self.q_table[state])
    
    def learn(self, state, action, reward, next_state):
        old_value = self.q_table[state, action]
        next_max = np.max(self.q_table[next_state])
        new_value = old_value + self.alpha*(reward + self.gamma*next_max - old_value)
        self.q_table[state, action] = new_value

In [32]:
from tqdm import tqdm
from tqdm.notebook import trange
import random

In [33]:
def encode_location(location):
    if location == 'R': return 0
    elif location == 'G': return 1
    elif location == 'Y': return 2
    else: return 3

score_hitory = []
best_score = float('-inf')
agent = Agent(env)
epochs = 10001
for epoch in trange(epochs):
    terminated, truncated = False, False
    state, _ = env.reset()
    #state = ((taxi_row * 5 + taxt_col) * 5 + passenger_location)* 4 + destination
    #env.env.s = state
    #info = env.decode(state)
    score = 0
    
    while not (terminated or truncated):
        if random.uniform(0, 1) < agent.epsilon:
            action = env.action_space.sample(info['action_mask'])
        else:
            action = agent.choose_action(state)
        new_state, reward, terminated, truncated, info = env.step(action)
        
        agent.learn(state, action, reward, new_state)
        state = new_state
        score += reward
    if score >= best_score:
        best_score = score
        np.save('q_table2.npy', agent.q_table)
        
    score_hitory.append(score)
print(best_score)

  0%|          | 0/10001 [00:00<?, ?it/s]

15


In [34]:
def strategy(taxi_row, taxi_col, passenger, des):
    passenger_location = encode_location(passenger)
    destination = encode_location(des)
    agent.q_table = np.load('q_table2.npy')
    env_video = []
    list_action = []
    terminated, truncated = False, False
    state, info = env.reset()
    state = env.encode(taxi_row, taxi_col, passenger_location, destination)
    env.unwrapped.s = state
    score = 0
    while not (terminated or truncated):
        action = agent.choose_action(state)
        env_video.append(env.render())
        state, reward, terminated, truncated, info = env.step(action)

        score += reward
        list_action.append(action)
    print(score)
    with VideoWriter(fps = 5) as vw:
        for frame in env_video:
            vw.add(frame)
    print(list_action)

In [35]:
strategy(3, 1,'R', 'B')

8


[1, 1, 1, 3, 4, 0, 0, 2, 2, 2, 0, 0, 5]
